<a href="https://colab.research.google.com/github/bicks1/hughesintern/blob/main/gff_gene_representative_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**File Name**:

```
gff_gene_representative.v3.ipynb
```

**Description**:

```
This program is a part of a series of programs for information extraction and mining of gene annotations in GFF3 files.

Using this script, exonic and intronic gene information is extracted for individual genes.

Feature/Type (column 3) defintions: http://www.sequenceontology.org/browser/obob.cgi

Biotype(attribute in column 9) definition: https://www.gencodegenes.org/pages/biotypes.html

```

**Authors**:

```
Sophia Bick, Chun Liang
```


###[Step 1]: Install Python modules, Map Google Drive that contains GFF3 files

In [ ]:
!pip install gffutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hg38gff = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/Homo_sapiens.GRCh38.109.chr.gff3"

In [ ]:
import gffutils

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# It takes a long time (about 8 mins) to build the database that contains parent (genes) and child (transcripts) relationships
# The following command imports the file into a local sqlite3 file-based database ("test.db")
##db = gffutils.create_db(hg38gff, dbfn='test.db', force=True, keep_order=False, merge_strategy='create_unique', sort_attribute_values=False)


In [ ]:
database = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/test.db"

In [ ]:
# The following command connects to the database previously created
# FeatureDB methods allow interaction with the database
db = gffutils.FeatureDB(database, keep_order=False)



---



###rep_csv
* uses MANE_Select or Ensembl_canonical for representative transcript
* includes longest, shortest, least exon, and most exon transcripts


In [ ]:
def rep_csv(db, gene_cat, filename):

  import csv


  # write csv file for the gene category specfied
  with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    fields = ["chromosome", "strand", "gene_name", "gene_id", "number_of_transcripts", "shortest_transcript", "shortest_length", "longest_transcript", "longest_length", "least_exon_transcript", "least_exon_number", "most_exon_transcript","most_exon_number", "representative_transcript", "representative_defined_by"]
    writer.writerow(fields)  # write header of csv
    for gene in db.features_of_type(gene_cat):  # for ea featuretype gene
      # transcript initialize
      counter = 0
      trans_min = 9999999999
      trans_max = 0

      # exon initialize
      exon_min = 9999999999
      exon_max = 0

      key = gene["gene_id"][0]  # key is gene's gene_id (internaly 1 item list)
      #print(key)

      # each transcript/child of gene
      #### CHANGE featuretype= if want to change type of transcripts analyzed
      for c in db.children(gene, featuretype=('mRNA', 'lnc_RNA', 'transcript', 'pseudogenic_transcript', 'ncRNA', 'snRNA', 'miRNA', 'unconfirmed_transcript', 'snoRNA', 'V_gene_segment', 'J_gene_segment', 'scRNA', 'rRNA', 'D_gene_segment', 'C_gene_segment', 'tRNA')):
        exon_count = 0  # flush exon count for ea new transcript
        counter += 1 # count num of transcripts in gene
        length = abs(c.end - c.start) + 1  # length of transcript (nt)

        for exon in db.children(c, featuretype="exon"):
          exon_count += 1  # number of exons in ea transcript
          #print(exon)

        try:
          for ea in c["tag"]:
            if ea == "Ensembl_canonical":
              represent = c["transcript_id"][0]
              represent_defined = ea
            elif ea == "MANE_Select":
              if represent == c["transcript_id"][0]:
                represent_defined = represent_defined + ':' + ea
              else:
                print("MANE and Ensembl canonical tags not on same transcript, MANE is on transcript", c["transcript_ic"][0], "and Ensembl canonical is on", represent)
        except KeyError:
          #print("no tag")
          pass

        category = c.featuretype

        if length < trans_min:  # determine min and max length and transcript ID
          trans_min = length
          trans_min_label = c["transcript_id"][0]  # changed from "ID"
          trans_min_type = c.featuretype
        elif length == trans_min:
          trans_min_label = trans_min_label + ':' + c["transcript_id"][0]
        if length > trans_max:
          trans_max = length
          trans_max_label = c["transcript_id"][0]
          trans_max_type = c.featuretype
        elif length == trans_max:
          trans_max_label = trans_max_label + ':' + c["transcript_id"][0]

        if exon_count < exon_min:  # determine min and max exon count and transcript ID
          exon_min = exon_count
          exon_min_label = c["transcript_id"][0]
        elif exon_count == exon_min:
          exon_min_label = exon_min_label + ':' + c["transcript_id"][0]

        if exon_count > exon_max:
          exon_max = exon_count
          exon_max_label = c["transcript_id"][0]
        elif exon_count == exon_max:
          exon_max_label = exon_max_label + ':' + c["transcript_id"][0]

      try:
          gene_name = gene["Name"][0]
      except KeyError:
          gene_name = "None"

      #print(counter)
      writer.writerow([gene.seqid, gene.strand, gene_name, gene["gene_id"][0], counter, trans_min_label, trans_min, trans_max_label, trans_max, exon_min_label, exon_min, exon_max_label, exon_max, represent, represent_defined])
      #print(gene.seqid, gene.strand, gene_name, gene["gene_id"][0], trans_min_label, trans_min,  trans_max_label, trans_max, exon_min_label, exon_min, exon_max_label, exon_max, represent, represent_defined)

In [ ]:
rep_csv(db, ("gene", "ncRNA_gene", "pseudogene"), "gff_gene_represent_all.csv")

In [ ]:
df = pd.read_csv("gff_gene_represent_all.csv")
df.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all.csv")

In [ ]:
print(len(df))

62656


Code to make separate files for longest, shortest, least exon, and most exon transcript

In [ ]:
df1 = df.iloc[: , [0, 1, 2, 3, 4, 5, 6]].copy()
print(df1)
df1.to_csv("gff_gene_represent_all_shortest.csv")
df1.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all_shortest.csv")

In [ ]:
df2 = df.iloc[: , [0, 1, 2, 3, 4, 7, 8]].copy()
print(df2)
df2.to_csv("gff_gene_represent_all_longest.csv")
df2.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all_longest.csv")

In [ ]:
df3 = df.iloc[: , [0, 1, 2, 3, 4, 9, 10]].copy()
print(df3)
df3.to_csv("gff_gene_represent_all_leastexon.csv")
df3.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all_leastexon.csv")

In [ ]:
df4 = df.iloc[: , [0, 1, 2, 3, 4, 11, 12]].copy()
print(df4)
df4.to_csv("gff_gene_represent_all_mostexon.csv")
df4.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all_mostexon.csv")

In [ ]:
df5 = df.iloc[: , [0, 1, 2, 3, 4, 13, 14]].copy()
print(df5)
df5.to_csv("gff_gene_represent_all_representative.csv")
df5.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all_representative.csv")

### consensus_rep
* detects if gene has a transcript with exact consensus match or not

In [ ]:
def consensus_rep(db, consensus_file, new_file):
  import csv
  with open(new_file, 'w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    fields = ["chromosome", "strand", "gene_name", "gene_id", "gene_start", "gene_end","number_of_transcripts", "exact_consensus_matched_transcript"]
    writer.writerow(fields)
    with open(consensus_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile)
      next(f_reader)
      for row in f_reader:
        chrom = row[1]
        strand = row[2]
        name = row[3]
        gene_id = row[4]
        g_start = row[5]
        g_end = row[6]
        exon_num = row[7]
        consensus_exons = row[9]
        consensus_dict = {}
        consensus_exact = "None"
        transcript_count = 0
        #print(consensus_exons.split(";"))
        for block in consensus_exons.split(";"):
          #print(block.split(":"))
          consensus_dict[block.split(":")[0]] = block.split(":")[1]
        #print(consensus_dict)
        consensus_dict_len = len(consensus_dict)

        if strand == "+":  # for +, exons must ordered by start, ascending (smallest start first, then more)
          order_by = ("start", "end")
          reverse = False
        elif strand == "-":  # for -, exons must ordered by end, descending (greatest end first, then less)
          order_by = ("end")
          reverse = True

        for transcript in db.children("gene:"+gene_id, featuretype=('mRNA', 'lnc_RNA', 'transcript', 'pseudogenic_transcript', 'ncRNA', 'snRNA', 'miRNA', 'unconfirmed_transcript', 'snoRNA', 'V_gene_segment', 'J_gene_segment', 'scRNA', 'rRNA', 'D_gene_segment', 'C_gene_segment', 'tRNA')):
          #print(transcript)
          transcript_count +=1
          exon_counter = 1
          for exon in db.children(transcript, featuretype="exon", order_by=order_by, reverse=reverse):
            #print(strand)
            #print(exon.start)
            #print(exon.end)
            if strand == "+":  # determine the coordinates of the current exon in the gene's current transcript
              exon_coord = str(exon.start) + '-' + str(exon.end)
            elif strand == "-":
              exon_coord = str(exon.end) + '-' + str(exon.start)
            #print(exon_coord)
            if exon_coord == consensus_dict["ConsensusExon_" + str(exon_counter)]:  # ex: if 3rd exon of current transcript and 3rd consensus exon match...
              if exon_counter == consensus_dict_len:  # if at last comparison of current exon
                #print("Exact match", transcript["transcript_id"][0]) # transcript is exact to consensus exons
                consensus_exact = transcript['transcript_id'][0]
              else:  # not at last exon/consensus comparison yet
                exon_counter += 1
            else:  # exon coordinates do not match the consensus exon's coordinates
              #print("not exact, breaking")
              break

        writer.writerow([chrom, strand, name, gene_id, g_start, g_end, transcript_count, consensus_exact])
        #print([chrom, strand, name, gene_id, g_start, g_end, consensus_exact])



In [ ]:
csvfile2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_all.csv"

In [ ]:
consensus_rep(db, csvfile2, "gff_gene_represent_exactconsensus.csv")

In [ ]:
df = pd.read_csv("gff_gene_represent_exactconsensus.csv")
df.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_exactconsensus.csv")

In [ ]:
print(len(df))

62656


### Various algorithms to determine representative transcript
* algorithm 1 and 2 uses different ratios of exonic bp. Ties for ratios are broken by MANE_Select tag (or Ensembl_canonical tag if MANE not available)

### consensus_rep_a1
algorithm 1: (sum of each transcript exons' length) / (gene's length)

highest ratio is representative transcript

can only be 1.0 if gene has one transcript with one exon
* rep transcript has most exon cover for total gene length
* gene length is introns and exons

In [ ]:
def consensus_rep_a1(db, consensus_file, new_file):
  import csv
  with open(new_file, 'w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    fields = ["chromosome", "strand", "gene_name", "gene_id", "gene_start", "gene_end","number_of_transcripts", "representative_transcript", "ratio"]
    writer.writerow(fields)
    with open(consensus_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile)
      next(f_reader)
      for row in f_reader:
        chrom = row[1]
        strand = row[2]
        name = row[3]
        gene_id = row[4]
        g_start = int(row[5])
        g_end = int(row[6])
        exon_num = row[7]
        consensus_exons = row[9]
        transcript_count = 0

        gene_length = abs(g_end - g_start) + 1

        high_ratio = 0
        for transcript in db.children("gene:"+gene_id, featuretype=('mRNA', 'lnc_RNA', 'transcript', 'pseudogenic_transcript', 'ncRNA', 'snRNA', 'miRNA', 'unconfirmed_transcript', 'snoRNA', 'V_gene_segment', 'J_gene_segment', 'scRNA', 'rRNA', 'D_gene_segment', 'C_gene_segment', 'tRNA')):
          #print(transcript)
          transcript_count +=1  # count num of transcripts
          exon_counter = 1
          exonic_bp = db.children_bp(transcript, child_featuretype="exon")  # gather len of all exons in that transcript
          transcript_ratio = exonic_bp/gene_length  # save current transcript's ratio
          if transcript_ratio > high_ratio:  # if current transcript's ratio is higher than previously saved ratio
            high_ratio = transcript_ratio  # save new high ratio
            r_transcript = transcript["transcript_id"][0]  # save transcript id
          elif transcript_ratio == high_ratio:  # if there is a tie
            try:  # use MANE_Select &/or Ensembl_canonical to break tie
              if "MANE_Select" in list(transcript["tag"]):
                r_transcript = transcript["transcript_id"][0]
              elif "Ensembl_canonical" in list(transcript["tag"]):
                r_transcript = transcript["transcript_id"][0]
              else:
                if "MANE_Select" in db["transcript:" + r_transcript]["tag"]:
                  pass
                elif "Ensembl_canonical" in db["transcript:" + r_transcript]["tag"]:
                  pass
                else:
                  #print("cannot break tie based on MANE or Ensembl_canonical")
                  pass
            except KeyError:
              #print("no tag")
              pass

          for exon in db.children(transcript, featuretype="exon"):
            exon_counter +=1

        if high_ratio > 1.0:
          print("############potential error!")

        writer.writerow([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio])
        #print([chrom, strand, name, gene_id, str(g_start), str(g_end), transcript_count, r_transcript, high_ratio])


In [ ]:
csvfile2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_all.csv"

In [ ]:
consensus_rep_a1(db, csvfile2, "gff_gene_representative_algorithm1.csv")

In [ ]:
df = pd.read_csv("gff_gene_representative_algorithm1.csv")
df.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_representative_algorithm1.csv")

### consensus_rep_a2
algorithm 2: (sum of each transcript's exons' length) / (length of gene's consensus exons)

Transcript with highest ratio is representative transcript for gene

can only be 1.0 if transcript has all possible exons in it
* rep transcript has most amount of exon cover compared to total amount of possible exons across all transcripts in gene

In [ ]:
def consensus_rep_a2(db, consensus_file, new_file):
  import csv
  with open(new_file, 'w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    fields = ["chromosome", "strand", "gene_name", "gene_id", "gene_start", "gene_end", "number_of_transcripts", "representative_transcript", "ratio"]
    writer.writerow(fields)
    with open(consensus_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile)
      next(f_reader)
      for row in f_reader:
        chrom = row[1]
        strand = row[2]
        name = row[3]
        gene_id = row[4]
        g_start = int(row[5])
        g_end = int(row[6])
        exon_num = row[7]
        consensus_exons = row[9]
        consensus_dict = {}
        transcript_count = 0
        consensus_exon_len = 0

        #print(consensus_exons.split(";"))
        for block in consensus_exons.split(";"):
          #print(block.split(":"))
          consensus_dict[block.split(":")[0]] = block.split(":")[1]
        #print(consensus_dict)
        for key in consensus_dict.keys():  # take len of ea consensus exon and add to total sum
          #print(consensus_dict[key].split("-")[1])
          #print(consensus_dict[key].split("-")[0])
          consensus_exon_len += abs(int(consensus_dict[key].split("-")[1]) - int(consensus_dict[key].split("-")[0])) + 1
        consensus_dict_len = len(consensus_dict)  # number of total consensus exons for ea gene

        high_ratio = 0
        for transcript in db.children("gene:"+gene_id, featuretype=('mRNA', 'lnc_RNA', 'transcript', 'pseudogenic_transcript', 'ncRNA', 'snRNA', 'miRNA', 'unconfirmed_transcript', 'snoRNA', 'V_gene_segment', 'J_gene_segment', 'scRNA', 'rRNA', 'D_gene_segment', 'C_gene_segment', 'tRNA')):
          #print(transcript)
          transcript_count +=1
          exon_counter = 1
          exonic_bp = db.children_bp(transcript, child_featuretype="exon")
          transcript_ratio = exonic_bp/consensus_exon_len
          if transcript_ratio > high_ratio:
            high_ratio = transcript_ratio
            r_transcript = transcript["transcript_id"][0]
          elif transcript_ratio == high_ratio:
            # r_transcript = r_transcript + ':' + transcript["transcript_id"][0]  # if you want to record ties
            try:  # use MANE_Select &/or Ensembl_canonical to break tie
              if "MANE_Select" in list(transcript["tag"]):
                r_transcript = transcript["transcript_id"][0]
              elif "Ensembl_canonical" in list(transcript["tag"]):
                r_transcript = transcript["transcript_id"][0]
              else:
                if "MANE_Select" in db["transcript:" + r_transcript]["tag"]:
                  pass
                elif "Ensembl_canonical" in db["transcript:" + r_transcript]["tag"]:
                  pass
                else:
                  #print("cannot break tie based on MANE or Ensembl_canonical")
                  pass
            except KeyError:
              #print("no tag")
              pass


          for exon in db.children(transcript, featuretype="exon"):
            exon_counter += 1

        if high_ratio > 1.0:
          print("############potential error!")

        #print(exonic_bp, "*******", consensus_exon_len)
        writer.writerow([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio ])
        #print([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio ])



In [ ]:
csvfile2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_all.csv"

In [ ]:
consensus_rep_a2(db, csvfile2, "gff_gene_representative_algorithm2.csv")

In [ ]:
df = pd.read_csv("gff_gene_representative_algorithm2.csv")
df.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_representative_algorithm2.csv")

### Comparison of algorithm 1, 2, and MANE_select tag

* take representative transcript columns from 3 files (a1, a2, mane)
* pool in a summary file
* see if each method returns same or different representative transcripts


In [ ]:
a1 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_representative_algorithm1.csv"
a2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_representative_algorithm2.csv"
mane = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_all.csv"

In [ ]:
df_a1 = pd.read_csv(a1)
df_a2 = pd.read_csv(a2)
df_mane = pd.read_csv(mane)

In [ ]:
print(df_mane.columns)

In [ ]:
#print(df_a1.rename(columns={"representative_transcript":"representative_algorithm1"}))
df_all = pd.DataFrame()
df_all[["chromosome", "strand", "gene_name", "gene_id","gene_start", "gene_end", "number_of_transcripts"]] = df_a1[["chromosome", "strand", "gene_name", "gene_id", "gene_start", "gene_end", "number_of_transcripts"]]
df_all["representative_a1"] = df_a1["representative_transcript"]  #make column that shows the representative transcript under algorithm 1 for all genes file
df_all["representative_a2"] = df_a2["representative_transcript"]  # "" but for algorithm 2
df_all["representative_mane"] = df_mane["representative_transcript"]  # "" but for MANE/Ensembl_canonical
df_all["algorithm_compare"] = np.where(df_all["representative_a1"] == df_all["representative_a2"], "same","different")  # column to compare if algorithm 1 and 2 return same representative transcript
df_all["mane_compare"] = np.where(df_all["representative_a1"] == df_all["representative_mane"], "same", "different") # column to compare if MANE/Ensembl_canonical and algorithm 1 return same representative transcript
print(df_all)
print(df_all["algorithm_compare"].value_counts()["same"])


In [ ]:
df_all.to_csv("gff_gene_represent_compare.csv")
df_all.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_represent_compare.csv")

In [ ]:
# algorithm 1 and 2 are 99% same
print(df_all["algorithm_compare"].value_counts()["same"] / len(df_all))

0.9988029877425945


In [ ]:
# algorithm 1 and mane are 81% the same
print(df_all["mane_compare"].value_counts()["same"] / len(df_all))

0.8166975229826353


### Printout of ties using algorithm 1 with no MANE Select tiebreaker

file with all genes that had a tie for representative transcript based on ratio of exonic bp to gene length

In [ ]:
def consensus_rep_old_a1(db, consensus_file, new_file):
  import csv
  with open(new_file, 'w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    fields = ["chromosome", "strand", "gene_name", "gene_id", "gene_start", "gene_end", "number_of_transcripts", "representative_transcript", "ratio"]
    writer.writerow(fields)
    with open(consensus_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile)
      next(f_reader)
      for row in f_reader:
        chrom = row[1]
        strand = row[2]
        name = row[3]
        gene_id = row[4]
        g_start = int(row[5])
        g_end = int(row[6])
        exon_num = row[7]
        consensus_exons = row[9]
        consensus_dict = {}
        transcript_count = 0
        #consensus_exon_len = 0
        gene_length = abs(g_end - g_start) + 1

        '''#print(consensus_exons.split(";"))
        for block in consensus_exons.split(";"):
          #print(block.split(":"))
          consensus_dict[block.split(":")[0]] = block.split(":")[1]
        #print(consensus_dict)
        for key in consensus_dict.keys():  # take len of ea consensus exon and add to total sum
          #print(consensus_dict[key].split("-")[1])
          #print(consensus_dict[key].split("-")[0])
          consensus_exon_len += abs(int(consensus_dict[key].split("-")[1]) - int(consensus_dict[key].split("-")[0])) + 1
        consensus_dict_len = len(consensus_dict)  # number of total consensus exons for ea gene'''

        high_ratio = 0
        high_exonic = 0
        for transcript in db.children("gene:"+gene_id, featuretype=('mRNA', 'lnc_RNA', 'transcript', 'pseudogenic_transcript', 'ncRNA', 'snRNA', 'miRNA', 'unconfirmed_transcript', 'snoRNA', 'V_gene_segment', 'J_gene_segment', 'scRNA', 'rRNA', 'D_gene_segment', 'C_gene_segment', 'tRNA')):
          #print(transcript)
          transcript_count +=1
          exon_counter = 1
          exonic_bp = db.children_bp(transcript, child_featuretype="exon")
          #transcript_ratio = (exonic_bp/gene_length)
          #if transcript_ratio > high_ratio:
            #high_ratio = transcript_ratio
            #r_transcript = transcript["transcript_id"][0]
          #elif transcript_ratio == high_ratio:
            #r_transcript = r_transcript + ':' + transcript["transcript_id"][0]

          if exonic_bp > high_exonic:
            high_exonic = exonic_bp
            r_transcript = transcript["transcript_id"][0]
          elif exonic_bp == high_exonic:
            r_transcript = r_transcript + ':' + transcript["transcript_id"][0]

          for exon in db.children(transcript, featuretype="exon"):
            exon_counter += 1

        high_ratio = high_exonic / gene_length

        if high_ratio > 1.0:
          print("############potential error!")

        #print(exonic_bp, "*******", consensus_exon_len)
        #if ":" in r_transcript:
          #writer.writerow([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio ])

        if ":" in r_transcript:
          writer.writerow([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio ])
        #print([chrom, strand, name, gene_id, g_start, g_end, transcript_count, r_transcript, high_ratio ])



In [ ]:
csvfile2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_all.csv"

In [ ]:
consensus_rep_old_a1(db, csvfile2, "representative_a1_ties_exonicbp.csv")